In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD

In [ ]:
def load_dataset(image_paths):
    data = []
    labels = []
    for i, path in enumerate(image_paths):
        if i > 0 and (i + 1) % 500 == 0:
            print(f"Loading {i+1} of {len(image_paths)} images...")
        image = cv2.imread(path)  # Load the image
        # Resize the image to a fixed size (ignore the aspect ratio)
        image = cv2.resize(image, (32, 32), interpolation=cv2.INTER_AREA)
        # Extract the class label
        label = path.split(os.path.sep)[-2]
        data.append(image)
        labels.append(label)
    data = np.array(data)
    labels = np.array(labels)
    return data, labels

In [ ]:
# Grab the list of images
dataset = os.path.expanduser("~/work/DL4CV/datasets/animals")
image_paths = glob.glob(dataset + "/**/*.jpg", recursive=True)

# Load the dataset
data, labels = load_dataset(image_paths)
print(data.shape)

# Scale the raw pixel intensities to the range [0, 1]
data = data.astype("float") / 255.0

# Display memory consumption
print(f"data matrix: {data.nbytes / (1024 * 1024.0):.1f} MB")

# Partition the data into training (75%) and testing (25%) splits
(trainX, testX, trainY, testY) = train_test_split(
    data, labels, test_size=0.25, random_state=42
)

# Convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [ ]:
# Define the architecture using Keras
height = 32
width = 32
depth = 3
classes = 3

model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same", input_shape=(height, width, depth)))
model.add(Activation("relu"))
model.add(Flatten())
model.add(Dense(classes))
model.add(Activation("softmax"))

# Initialize the optimizer and model
opt = SGD(learning_rate=0.005)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

# Train the model using SGD
EPOCHS = 100
H = model.fit(
    trainX,
    trainY,
    validation_data=(testX, testY),
    batch_size=32,
    epochs=EPOCHS,
    verbose=1,
)

In [ ]:
# Evaluate the network
predictions = model.predict(testX, batch_size=32)
report = classification_report(
    testY.argmax(axis=1),
    predictions.argmax(axis=1),
    target_names=[str(x) for x in lb.classes_],
)
print(report)

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()